In [1]:
import pandas as pd
import datetime as dt
import pytz
import os

In [3]:
def mod_concat_NEM_test(file_path):
    df = pd.read_pickle(file_path)
    df = df.set_index('PeriodID', drop=True)
    df.index = pd.to_datetime(df.index).tz_localize(None)
    df[["Price", "Increase"]] = df[["Price", "Increase"]].astype(float)
    print(df.info())
    df.sort_index(inplace=True, ascending=False)
    df.index = df.index - dt.timedelta(minutes = 5)
    df = df.loc[df.index < dt.datetime(2018,8,1), :]
    return df

In [4]:
df = mod_concat_NEM_test('{}/nem-data/nemde_clean/concat_for_energy_SA1.pkl'.format(os.environ['HOME']))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5951880 entries, 2015-06-23 13:40:00 to 2012-01-11 12:45:00
Data columns (total 9 columns):
RegionID            object
Market              object
Price               float64
Unit                object
DispatchedMarket    object
BandNo              object
Increase            float64
RRNBandPrice        object
BandCost            object
dtypes: float64(2), object(7)
memory usage: 454.1+ MB
None


In [5]:
df.head()

,RegionID,Market,Price,Unit,DispatchedMarket,BandNo,Increase,RRNBandPrice,BandCost
PeriodID,,,,,,,,,
2018-07-31 23:55:00,SA1,Energy,70.70204,ER01,ENOF,3,0.34712,53.75,18.657483
2018-07-31 23:55:00,SA1,Energy,70.70204,ER04,ENOF,3,0.31058,53.75,16.693537
2018-07-31 23:55:00,SA1,Energy,70.70204,ER03,ENOF,3,0.34712,53.75,18.657483
2018-07-31 23:55:00,SA1,Energy,70.70204,ER02,ENOF,3,0.31058,53.75,16.693537
2018-07-31 23:50:00,SA1,Energy,69.24109,DEVILS_G,L5MI,4,-1.29132,0.14,-0.180785


In [10]:
#final code_block <- works

def mod_concat_NEMDE(file_path):
    df = pd.read_pickle(file_path)
    df.drop(columns=["BandNo","RRNBandPrice","BandCost"], inplace=True)
    df = df.set_index('PeriodID', drop=True)
    df.index = pd.to_datetime(df.index).tz_localize(None)
    df[["Price", "Increase"]] = df[["Price", "Increase"]].astype(float)
    df.sort_index(inplace=True, ascending=False)
    df.index = df.index - dt.timedelta(minutes = 5)
    df = df.loc[df.index < dt.datetime(2018,8,1), :]
    assert df["Price"].dtype == float
    assert df["Increase"].dtype == float
    assert df.isna().any().any() == False
    return df

In [8]:
file = '{}/nem-data/nemde_clean/concat_for_energy_SA1'.format(os.environ['HOME'])

df = mod_concat_NEMDE(file)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5951880 entries, 2015-06-23 13:40:00 to 2012-01-11 12:45:00
Data columns (total 6 columns):
RegionID            object
Market              object
Price               float64
Unit                object
DispatchedMarket    object
Increase            float64
dtypes: float64(2), object(4)
memory usage: 317.9+ MB
None


In [9]:
print(df.shape)
df.head()

(5951707, 6)


,RegionID,Market,Price,Unit,DispatchedMarket,Increase
PeriodID,,,,,,
2018-07-31 23:55:00,SA1,Energy,70.70204,ER01,ENOF,0.34712
2018-07-31 23:55:00,SA1,Energy,70.70204,ER04,ENOF,0.31058
2018-07-31 23:55:00,SA1,Energy,70.70204,ER03,ENOF,0.34712
2018-07-31 23:55:00,SA1,Energy,70.70204,ER02,ENOF,0.31058
2018-07-31 23:50:00,SA1,Energy,69.24109,DEVILS_G,L5MI,-1.29132


In [11]:
df_grouped = df.groupby("PeriodID").agg({"Price" : "nunique"})
print("The number of time periods is {}.".format(df_grouped.shape[0]))
df_grouped.Price.value_counts()

The number of time periods is 946847.


1    946844
2         3
Name: Price, dtype: int64

In [12]:
df_grouped.loc[df_grouped.Price == 2]

,Price
PeriodID,
2015-02-22 23:30:00,2
2015-05-01 08:40:00,2
2016-10-31 20:30:00,2


In [13]:
df_price_count = df.groupby("PeriodID").agg({"Price" : "count"}).sort_values(by="PeriodID", ascending=False)

df_price_count[(df_price_count.index == dt.datetime(2015,2,22,23,30,0)) | 
   (df_price_count.index == dt.datetime(2015,5,1,8,40,0)) | 
   (df_price_count.index == dt.datetime(2016,10,31,20,30,0))]

,Price
PeriodID,
2016-10-31 20:30:00,15
2015-05-01 08:40:00,41
2015-02-22 23:30:00,8


In [14]:
df_outlier = df[(df.index == dt.datetime(2015,2,22,23,30,0)) | 
   (df.index == dt.datetime(2015,5,1,8,40,0)) | 
   (df.index == dt.datetime(2016,10,31,20,30,0))]

df_outlier.groupby('PeriodID').agg({'Price' : ['min', 'max']})

Price              
                             min           max
PeriodID                                      
2015-02-22 23:30:00  97591.06624  368100.07478
2015-05-01 08:40:00     70.29670      72.09740
2016-10-31 20:30:00     95.98007  509229.65442

In [15]:
df_group_period = df.groupby("PeriodID").agg({"Price":"mean"}).sort_values(by="PeriodID",ascending=False)
df_group_period

,Price
PeriodID,
2018-07-31 23:55:00,70.70204
2018-07-31 23:50:00,69.24109
2018-07-31 23:45:00,69.94850
2018-07-31 23:40:00,74.04497
2018-07-31 23:35:00,76.44245
...,...
2009-07-01 04:20:00,1.75480
2009-07-01 04:15:00,1.44014
2009-07-01 04:10:00,-0.00006


In [207]:
pd.to_pickle(df_group_period, '{}/nem-data/nemde_clean/df_grouped_price.pkl'.format(os.environ['HOME']))

In [32]:
df_dispatch = pd.read_csv('/home/bastian/nem-data/clean/dispatch_price.csv', index_col = 0, parse_dates=True)
df_dispatch.sort_index(ascending=False, inplace=True)
df_dispatch = df_dispatch.loc[(df_dispatch.index <= df.index.max()) &
                              (df_dispatch.index >= df.index.min()), :][['RRP SA1']]
assert df_dispatch["RRP SA1"].dtype == float
assert df_dispatch.index.min() == df.index.min()
assert df_dispatch.index.max() == df.index.max()
print("The number of time periods is {}.".format(df_dispatch.shape[0]))
df_dispatch.head()

The number of time periods is 955536.


,RRP SA1
tempus_interval_time,
2018-07-31 23:55:00,70.70204
2018-07-31 23:50:00,69.24109
2018-07-31 23:45:00,69.94850
2018-07-31 23:40:00,74.04497
2018-07-31 23:35:00,76.44245


In [18]:
df_join = df_group_period.join(df_dispatch, how='inner')
print(df_join.shape)
df_join.head()

(946847, 2)


,Price,RRP SA1
2018-07-31 23:55:00,70.70204,70.70204
2018-07-31 23:50:00,69.24109,69.24109
2018-07-31 23:45:00,69.94850,69.94850
2018-07-31 23:40:00,74.04497,74.04497
2018-07-31 23:35:00,76.44245,76.44245


In [36]:
df_join

,Price,RRP SA1
2018-07-31 23:55:00,70.70204,70.70204
2018-07-31 23:50:00,69.24109,69.24109
2018-07-31 23:45:00,69.94850,69.94850
2018-07-31 23:40:00,74.04497,74.04497
2018-07-31 23:35:00,76.44245,76.44245
...,...,...
2009-07-01 04:20:00,1.75480,1.75480
2009-07-01 04:15:00,1.44014,1.44014
2009-07-01 04:10:00,-0.00006,-0.00006
2009-07-01 04:05:00,-0.00006,-0.00006


In [88]:
assert not df_join.all().all()

In [84]:
np.logical_and()

numpy.bool_

In [54]:
df_join

,Price,RRP SA1
2018-07-31 23:55:00,70.70204,70.70204
2018-07-31 23:50:00,69.24109,69.24109
2018-07-31 23:45:00,69.94850,69.94850
2018-07-31 23:40:00,74.04497,74.04497
2018-07-31 23:35:00,76.44245,76.44245
...,...,...
2009-07-01 04:20:00,1.75480,1.75480
2009-07-01 04:15:00,1.44014,1.44014
2009-07-01 04:10:00,-0.00006,-0.00006
2009-07-01 04:05:00,-0.00006,-0.00006


In [22]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 946847 entries, 2018-07-31 23:55:00 to 2009-07-01 04:00:00
Data columns (total 2 columns):
Price      946847 non-null float64
RRP SA1    946847 non-null float64
dtypes: float64(2)
memory usage: 21.7 MB


In [23]:
print("The percentage of mismatching dispatch prices from MMSDM and NEMDE is {} %.".format(round(((sum(abs((df_join["Price"] - df_join["RRP SA1"])\
                                                                                  /df_join["RRP SA1"]) > 0.000001)/df_join.shape[0])*100),2)))

The percentage of mismatching dispatch prices from MMSDM and NEMDE is 0.41 %.


In [24]:
df_join_mismatch = df_join.loc[abs((df_join["Price"] - df_join["RRP SA1"])/df_join["RRP SA1"]) > 0.000001, :]
print((abs(df_join_mismatch["Price"] - df_join_mismatch["RRP SA1"])).agg(["min", "max", "mean"]))
print(df_join_mismatch.shape)
df_join_mismatch.head()

min     1.010000e-03
max     2.099937e+06
mean    8.669131e+03
dtype: float64
(3921, 2)


,Price,RRP SA1
2018-04-27 18:50:00,426000.00401,14200.00
2018-04-27 18:45:00,426000.00403,14200.00
2018-04-26 11:30:00,426000.00385,14200.00
2018-04-26 11:25:00,426000.00411,14200.00
2018-02-08 01:00:00,14673.29560,590.01


In [25]:
df_join.drop(df_join_mismatch.index, inplace=True)
print(df_join.shape)

(942926, 2)


In [26]:
df_join.info()
df_join.loc[abs(((df_join["Price"] - df_join["RRP SA1"])/df_join["RRP SA1"])) > 0.000001]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 942926 entries, 2018-07-31 23:55:00 to 2009-07-01 04:00:00
Data columns (total 2 columns):
Price      942926 non-null float64
RRP SA1    942926 non-null float64
dtypes: float64(2)
memory usage: 21.6 MB


,Price,RRP SA1


In [31]:
(abs(((df_join["Price"] - df_join["RRP SA1"])/df_join["RRP SA1"])) > 0.000001).any()

False

In [206]:
assert df_group_price.shape[0] - df_join_mismatch.shape[0] == df_join.shape[0]
assert sum(abs((df_join["Price"] - df_join["RRP SA1"])/df_join["RRP SA1"]) > 0.000001) == 0
print(df_join.shape)
df_join.head() #cleeeaaan

(942926, 2)


,Price,RRP SA1
2018-07-31 23:55:00,70.70204,70.70204
2018-07-31 23:50:00,69.24109,69.24109
2018-07-31 23:45:00,69.94850,69.94850
2018-07-31 23:40:00,74.04497,74.04497
2018-07-31 23:35:00,76.44245,76.44245


In [205]:
df_join.to_pickle('{}/nem-data/nemde_clean/marginal_DUID_concat_dispatch.pkl'.format(os.environ['HOME']))

In [132]:
df_join_mismatch.groupby([df_join_mismatch.index.year, df_join_mismatch.index.month]).mean()

Price       RRP SA1
2009 9   5.635385e+04  10000.000000
2010 2   2.174719e+04  10000.000000
     6   3.231561e+04  10000.000000
     7   6.625955e+04     43.770300
     11  2.343915e+01     22.765203
2011 1   1.434098e+04  12500.000000
     2   4.710340e+01     43.770150
     5   9.222582e+04  12500.000000
     7  -1.000080e+03  -1000.000000
     9   3.918098e+01     38.770130
     10  1.591242e+04  12500.000000
     11  4.618960e+05  12500.000000
2012 7   4.606830e+05   7545.834620
     11  2.016406e+04  12900.000000
     12 -2.680101e+04  -1000.000000
2013 1   4.298859e+05  12900.000000
     6   1.307892e+04  12900.000000
     12  1.910566e+04  13100.000000
2014 1   1.338380e+04  13100.000000
2015 2   1.576961e+06  13494.500000
     5   7.056022e+01     70.296700
     8   1.380099e+04  13800.000000
     9   2.070000e+06  12301.300000
     12  1.402568e+04  13800.000000
2016 2  -1.067701e+03  -1000.000000
     3   1.202903e+06  13004.662450
     4  -1.045348e+03  -1000.000000
     7   3.290066e+04  14000.000000
     9   9.325866e+03     90.372990
     10  4.507455e+02     64.854599
     11  2.100000e+06  14000.000000
     12  3.425576e+05  14000.000000
2017 2   1.123866e+06  14000.000000
     3   1.400117e+06   9374.996687
     5   2.100000e+06  14000.000000
2018 1   1.875223e+04  14200.000000
     2   1.467330e+04    590.010000
     4   4.260000e+05  14200.000000

In [133]:
df_join_mismatch.shape

(1434, 2)

In [ ]:
df1 = df.loc[df.PeriodID < pytz.FixedOffset(600).localize(dt.datetime(2018, 8, 1)), :]

In [ ]:
df.PeriodID = df.PeriodID.dt.tz_localize(None)
df.head()